# Setup

In [1]:
from langchain.prompts import PromptTemplate
from langchain import hub
from langchain.agents import Tool, AgentExecutor, initialize_agent, create_react_agent
from langchain_community.tools import GoogleSerperRun
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_openai import ChatOpenAI
import yaml
import os

In [2]:
with open('config.yaml', 'r') as config_file:
    config = yaml.safe_load(config_file)
os.environ['OPENAI_API_KEY'] = config['OPENAI_API_KEY']
os.environ['SERPER_API_KEY'] = config['SERPER_API_KEY']

In [ ]:
openai = ChatOpenAI(model_name='gpt-4o-mini', temperature=0)

# Assitente Financeiro Pessoal

In [21]:
prompt = '''
Como assistente financeiro pessoal, ajude a responder as seguintes perguntas com ajuda da internet.
Perguntas: {q}
'''
prompt_template = PromptTemplate.from_template(prompt)
print(prompt_template)

input_variables=['q'] template='\nComo assistente financeiro pessoal, ajude a responder as seguintes perguntas com ajuda da internet.\nPerguntas: {q}\n'


In [22]:
react_instructions  = hub.pull('hwchase17/react')
print(react_instructions )

input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'] metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'} template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}'


In [23]:
# Ferramenta 1 Python REPL
python_repl = PythonREPLTool()
python_repl_tool = Tool(
    name='Python REPL',
    func=python_repl.run,
    description='''Qualquer tipo de cálculo deve usar esta ferramenta. Você não deve realizar
                    o cálculo diretamente. Você deve inserir código Python.'''
)

In [24]:
# Ferramenta  2 busca Google
api_wrapper = GoogleSerperAPIWrapper(
    gl="br",
    hl="pt-br",
)

google_search = GoogleSerperRun(api_wrapper=api_wrapper)

google_tool = Tool(
    name='Busca Google',
    func=google_search.run,
    description='''Útil para encontrar informações e dicas de economia e opções de investimento.
                Você sempre deve pesquisar na internet as melhores dicas usando esta ferramenta, não
                responda diretamente. Sua resposta deve informar que há elementos pesquisados na internet'''
)

In [25]:
tools = [python_repl_tool, google_tool]
agent = create_react_agent(openai, tools, react_instructions)

In [26]:
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=5
)
print(agent_executor)

verbose=True agent=RunnableAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
})
| PromptTemplate(input_variables=['agent_scratchpad', 'input'], partial_variables={'tools': "Python REPL(tool_input: 'Union[str, Dict[str, Any]]', verbose: 'Optional[bool]' = None, start_color: 'Optional[str]' = 'green', color: 'Optional[str]' = 'green', callbacks: 'Callbacks' = None, *, tags: 'Optional[List[str]]' = None, metadata: 'Optional[Dict[str, Any]]' = None, run_name: 'Optional[str]' = None, run_id: 'Optional[uuid.UUID]' = None, config: 'Optional[RunnableConfig]' = None, tool_call_id: 'Optional[str]' = None, **kwargs: 'Any') -> 'Any' - Qualquer tipo de cálculo deve usar esta ferramenta. Você não deve realizar\n                    o cálculo diretamente. Você deve inserir código Python.\nBusca Google(tool_input: 'Union[str, Dict[str, Any]]', verbose: 'Optional[bool]' = None, start_color: 'Optional[str]' = 'green', color: 'O

In [17]:
question = """
Eu ganho R$4000 por mês mas o total de minhas despesas é de R$3800 mais 500 de aluguel.
Como posso ajustar meu orçamento para economizar dinheiro?
"""

In [10]:
question = """
Minha renda é de R$6000 por mês, o total de minhas despesas é de R$2500 mais 1000 de aluguel.
Quais dicas de investimento você me dá?
"""

In [27]:
output = agent_executor.invoke({
    'input': prompt_template.format(q=question)
})



> Entering new AgentExecutor chain...
Thought: Para ajudar a ajustar o orçamento e economizar dinheiro, preciso entender melhor a situação financeira atual, identificar onde é possível cortar gastos e sugerir estratégias para economizar. Vou pesquisar na internet dicas práticas para ajustar o orçamento pessoal com base em uma renda de R$4000 e despesas fixas de R$4300 (R$3800 + R$500 de aluguel), o que indica um déficit mensal.

Action: Busca Google
Action Input: "como ajustar orçamento pessoal para economizar dinheiro quando despesas são maiores que a renda"Organizar o orçamento não precisa ser complicado: a regra 50-30-20 propõe uma divisão simples da renda para equilibrar os gastos. O que é a regra 50-30-20? · 50% para despesas essenciais: moradia, alimentação, transporte, contas fixas. · 30% para desejos e lazer: viagens, ... ... renda ao seguir esse orçamento. FAÇA ISSO SEMPRE QUE RECEBER SEU SALÁRIO | Como organizar suas finanças e guardar dinheiro? 728K views · 7 ... Descubra 

In [12]:
print(output['input'])


Como assistente financeiro pessoal, ajude a responder as seguintes perguntas com ajuda da internet.
Perguntas: 
Minha renda é de R$6000 por mês, o total de minhas despesas é de R$2500 mais 1000 de aluguel.
Quais dicas de investimento você me dá?




In [19]:
print(output['output'])

Para ajustar seu orçamento e economizar dinheiro, considere as seguintes dicas: organize seus ganhos e gastos, crie um orçamento realista, reduza gastos desnecessários, planeje suas compras, estabeleça metas de economia, adote hábitos financeiros inteligentes, evite compras por impulso e busque maneiras de cortar custos fixos. Essas estratégias podem ajudá-lo a economizar e a ter um melhor controle financeiro.
